# Spam Detector

In [1]:
import os         # for filesystem access
import fnmatch    # for Unix filename pattern matching

In [2]:
LINGSPAM_BARE_DATASET_PATH = "datasets/lingspam_public/bare"

In [3]:
documents = []

In [ ]:
def is_spam_file_name(file_name):
    return fnmatch.fnmatchcase(file_name, 'spmsg*')

## Reading and Preprocessing Data

### Read all the emails in the ten folders & mark each email with spam/not-spam

In [4]:
for root, dirs, file_names in os.walk(LINGSPAM_BARE_DATASET_PATH):
    for file_name in fnmatch.filter(file_names, '*.txt'):
        with open(os.path.join(root, file_name), 'r') as file:
            document = (file_name, is_spam_file_name(file_name), file.read())
            documents = documents + [document]

In [5]:
documents_length = len(documents)

if documents_length > 0:
    print("✅ Read %i documents" % len(documents))
else:
    print("❌ Could not read any documents")

✅ Read 2893 documents


### 80% training and 20% testing

### Fit and transform the training emails and transform the testing emails using a CountVectorizer

## Scikit-Learn Classifiers
##### For each classifier, print the precision, recall and f-score on the testing data

### Multinomial Naive Bayes

### K Neighbors Classifier

### Random Forest Classifier
##### you can set random_state=0

## Classifying using Readability Features

Rather than using the whole text content of an email, some characteristic features can be extracted per email, that will be fed to the classifier. Extract some features.
The features are:

    a) F1: The number of sentences in an email.
    b) F2: The number of verbs in an email.
    c) F3: The number of words containing both numeric and alphabetical characters.
    d) F4: The number of words in an email that are found in the spam list.
    e) F5: The number of words in an email that have more than 3 syllables.
    f) F6: The average number of syllables of words in an email.
    g) F7: The number of spelling mistakes in an email.
    
For F2, you can find useful code in Lab Assignment 5 solution on the MET website.
For F4, you will be checking how many words in a given email are found in a spam word-list. The
word-list you will be using can be found here.
For F5 and F6, you can use the library Pyphen (with lang=’en_GB’). For F7, you can use the
library PyEnchant.

The steps are:

    a) Create a list for every feature, where every element is the feature value of a given email (or use a
    dictionary, key is feature name, value is feature list).
    b) Build a feature matrix (list of lists), where every row corresponds to an email, and every column
    corresponds to a feature value of this email.
    c) Feed the feature matrix and the labels to any of the sklearn classifiers.
    
On the MET website, you will find a file titled “feature-construction”. This is an example of building a
feature matrix (steps “a” and “b”). Note that this is just a sample, the documents and the features to be
extracted will be different in the project.

##### For the classifier, print the precision, recall and f-score on the testing data.